In [1]:
!nvidia-smi

Sun May 22 08:17:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install datasets adapter-transformers

     |████████████████████████████████| 346 kB 9.3 MB/s 
     |████████████████████████████████| 3.9 MB 68.0 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 212 kB 99.6 MB/s 
     |████████████████████████████████| 140 kB 73.1 MB/s 
     |████████████████████████████████| 1.1 MB 66.2 MB/s 
     |████████████████████████████████| 127 kB 78.4 MB/s 
     |████████████████████████████████| 6.6 MB 83.5 MB/s 
     |████████████████████████████████| 880 kB 60.3 MB/s 
     |████████████████████████████████| 596 kB 82.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 78.9 MB/s 
     |████████████████████████████████| 144 kB 78.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9e5955c1aa1534280e089eabdf01f97ad5167c0b5b36d434c0c29b0e1a64bf48
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d66029

In [ ]:
# pip install --upgrade --force-reinstall pyarrow

In [5]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [11]:
from datasets import Dataset, DatasetDict, load_dataset
import warnings
import numpy as np
import pandas as pd
import json
from transformers.adapters.composition import Fuse
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    AutoModelWithHeads,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    default_data_collator,
    PreTrainedTokenizerFast,
    AutoAdapterModel,
    AdapterConfig,
    AutoModelForQuestionAnswering,
)
from transformers import AdapterConfig
from transformers.adapters.composition import Stack

warnings.filterwarnings("ignore")
# import wandb
# wandb.init(mode="disabled")
# wandb.init(mode="offline")

## Load Dataset

In [12]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [13]:
# training data
language = 'de'
input_file_path = f'./drive/MyDrive/Colab Files/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
train.head(1)

,index,question,context,answer_start,text,c_id,answers
0,56db2120e7c41114004b4dce,Wer war der Präsident der alternativen Programmierung bei Fox?,"Die 8. Staffel wurde am 13. Januar 2009 uraufgeführt. Mike Darnell, der Präsident der alternativen Programmiersoftware von Fox, erklärte, die Saison werde sich mehr auf das persönliche Leben der Teilnehmer konzentrieren. Viel frühes Augenmerk in der Show konzentrierte sich daher auf die Witwerschaft von Danny Gokey.",54,Mike Darnell,0,"{'answer_start': [54], 'text': ['Mike Darnell']}"


In [14]:
# Validation data
# language = 'hi'
input_file_path = f'./drive/MyDrive/Colab Files/squad.translate.dev.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)
dev['answers'] = dev[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
dev.head(1)

,index,question,context,answer_start,text,c_id,answers
0,5730a951069b531400832213,"Jules Ferry dachte, dass die ""höheren Rassen"" eine Pflicht zu was haben?","Es wurde zu einer moralischen Rechtfertigung, die Welt durch das Christentum und die französische Kultur auf französische Standards zu bringen. Der führende Exponent des Kolonialismus, Jules Ferry, erklärte 1884, Frankreich habe eine zivilisierende Mission: ""Die höheren Rassen haben ein Recht auf die niederen Rassen, sie haben die Pflicht, die Untergebenen zu zivilisieren"". Es wurden volle Bürgerrechte - ""Assimilation"" - angeboten, obwohl die Assimilation in Wirklichkeit immer am Horizont lag. Im Gegensatz zu Großbritannien schickte Frankreich eine kleine Anzahl von Siedlern in seine Kolonien, mit der einzigen bemerkenswerten Ausnahme in Algerien, wo die französischen Siedler dennoch immer eine kleine Minderheit blieben.",342,die Untergebenen zu zivilisieren,0,"{'answer_start': [342], 'text': ['die Untergebenen zu zivilisieren']}"


In [15]:
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(dev)


squad_hi = DatasetDict()

squad_hi['train'] = tds
squad_hi['validation'] = vds

In [16]:
squad_hi

DatasetDict({
    train: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 82603
    })
    validation: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 32950
    })
})

In [18]:
# squad_en = load_dataset("squad")
# squad_en

## Create QA Features

In [19]:

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
assert isinstance(tokenizer, PreTrainedTokenizerFast)
pad_on_right = tokenizer.padding_side == "right"

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [20]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["index"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [21]:
# squad = squad.map(
#     prepare_train_features, batched=True, remove_columns=squad["train"].column_names
# )

# squad_en = squad_en.map(
#     prepare_train_features, batched=True, remove_columns=squad_en["train"].column_names
# )

squad_hi = squad_hi.map(
    prepare_train_features, batched=True, remove_columns=squad_hi["train"].column_names
)


  0%|          | 0/83 [00:00<?, ?ba/s]

  0%|          | 0/33 [00:00<?, ?ba/s]

## Load Model and Adapters

### Enable EN only for training

In [23]:
model = AutoModelForQuestionAnswering.from_pretrained(f'./drive/MyDrive/Colab Files/models/hi-adapter-en-trained')
config = AdapterConfig.load("pfeiffer",non_linearity="relu", reduction_factor=2)
adapter_name_1 = model.load_adapter("en/wiki@ukp", config=config,model_name='xlm-roberta-base')
adapter_name_2 = model.load_adapter("de/wiki@ukp",config=config,model_name='xlm-roberta-base')
task_adapter = model.load_adapter(f'./drive/MyDrive/Colab Files/models/hi-adapter-en-trained/pfeiffer_xlm_base', source="hf",load_as = 'pfeiffer_xlm_base')
model.set_active_adapters(adapter_name_2,task_adapter)
model.train_adapter('pfeiffer_xlm_base')

Some weights of the model checkpoint at ./drive/MyDrive/Colab Files/models/hi-adapter-en-trained were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.encoder.layer.3.output.adapters.hi.adapter_down.0.bias', 'roberta.encoder.layer.2.output.adapters.pfeiffer_xlm_base.adapter_down.0.bias', 'roberta.encoder.layer.5.output.adapters.pfeiffer_xlm_base.adapter_down.0.bias', 'roberta.encoder.layer.8.output.adapters.pfeiffer_xlm_base.adapter_down.0.bias', 'roberta.encoder.layer.0.output.adapters.hi.adapter_down.0.weight', 'roberta.encoder.layer.10.output.adapters.en.adapter_up.weight', 'roberta.encoder.layer.2.output.adapters.hi.adapter_up.weight', 'roberta.encoder.layer.8.output.adapters.pfeiffer_xlm_base.adapter_up.weight', 'roberta.encoder.layer.3.output.adapters.hi.adapter_up.weight', 'roberta.encoder.layer.2.output.adapters.en.adapter_up.weight', 'roberta.encoder.layer.7.output.adapters.hi.adapter_down.0.bias', 'roberta.encoder.layer.9.output.adapters.en.adapter_up.bias

Downloading:   0%|          | 0.00/28.2M [00:00<?, ?B/s]

Overwriting existing adapter 'pfeiffer_xlm_base'.


## Training



In [24]:
batch_size = 32

In [25]:
args = TrainingArguments(
    f"./{language}-adapter-{batch_size}",
    evaluation_strategy = "epoch",
    save_strategy= "epoch",
    learning_rate = 3e-5,
    warmup_ratio = 0.1,
    gradient_accumulation_steps = 8,
    num_train_epochs = 5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    seed=42,
    fp16=True,
    overwrite_output_dir=True,
    save_total_limit=1,
    push_to_hub=True,
    load_best_model_at_end=True)

In [26]:
data_collator = default_data_collator

In [28]:
trainer = Trainer(
    model,
    args,
    train_dataset=squad_hi["train"],
    eval_dataset=squad_hi["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

Cloning https://huggingface.co/subhasisj/de-adapter-32 into local empty directory.
Using amp half precision backend


In [30]:
# f'./drive/MyDrive/Colab Files/models/{language}-adapter-en-trained/all'

'./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all'

In [31]:
trainer.train()

***** Running training *****
  Num examples = 85981
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 1675


Epoch,Training Loss,Validation Loss
0,No log,5.703147
1,5.759200,5.570599


***** Running Evaluation *****
  Num examples = 34649
  Batch size = 32
Saving model checkpoint to ./de-adapter-32/checkpoint-335
Configuration saved in ./de-adapter-32/checkpoint-335/config.json
Model weights saved in ./de-adapter-32/checkpoint-335/pytorch_model.bin
tokenizer config file saved in ./de-adapter-32/checkpoint-335/tokenizer_config.json
Special tokens file saved in ./de-adapter-32/checkpoint-335/special_tokens_map.json
tokenizer config file saved in ./de-adapter-32/tokenizer_config.json
Special tokens file saved in ./de-adapter-32/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
***** Running Evaluation *****
  Num examples = 34649
  Batch size = 32
Saving model checkpoint to ./de-adapter-32/checkpoint-670
Configuration saved in ./de-adapter-32/checkpoint-670/config.json
Model weights saved in ./de-adapter-32/checkpoint-670/pytorch_model.bin
tokenizer config file saved in ./de-adapter-32/checkp

Epoch,Training Loss,Validation Loss
0,No log,5.703147
1,5.759200,5.570599
2,5.564700,5.489931
3,5.564700,5.448086
4,5.486500,5.434740


***** Running Evaluation *****
  Num examples = 34649
  Batch size = 32
Saving model checkpoint to ./de-adapter-32/checkpoint-1005
Configuration saved in ./de-adapter-32/checkpoint-1005/config.json
Model weights saved in ./de-adapter-32/checkpoint-1005/pytorch_model.bin
tokenizer config file saved in ./de-adapter-32/checkpoint-1005/tokenizer_config.json
Special tokens file saved in ./de-adapter-32/checkpoint-1005/special_tokens_map.json
Deleting older checkpoint [de-adapter-32/checkpoint-335] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 34649
  Batch size = 32
Saving model checkpoint to ./de-adapter-32/checkpoint-1340
Configuration saved in ./de-adapter-32/checkpoint-1340/config.json
Model weights saved in ./de-adapter-32/checkpoint-1340/pytorch_model.bin
tokenizer config file saved in ./de-adapter-32/checkpoint-1340/tokenizer_config.json
Special tokens file saved in ./de-adapter-32/checkpoint-1340/special_tokens_map.json
Deleting older checkpoint [de-ad

TrainOutput(global_step=1675, training_loss=5.586920293551772, metrics={'train_runtime': 8106.5498, 'train_samples_per_second': 53.032, 'train_steps_per_second': 0.207, 'total_flos': 9.15201153809879e+16, 'train_loss': 5.586920293551772, 'epoch': 5.0})

In [32]:
import os
os.makedirs(f'./drive/MyDrive/Colab Files/models/{language}-adapter-en-trained/all')

In [33]:
model.save_all_adapters(f'./drive/MyDrive/Colab Files/models/{language}-adapter-en-trained/all')

Configuration saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/en/adapter_config.json
Module weights saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/en/pytorch_adapter.bin
Configuration saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/en/head_config.json
Module weights saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/en/pytorch_model_head.bin
Configuration saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/hi/adapter_config.json
Module weights saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/hi/pytorch_adapter.bin
Configuration saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/hi/head_config.json
Module weights saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/hi/pytorch_model_head.bin
Configuration saved in ./drive/MyDrive/Colab Files/models/de-adapter-en-trained/all/pfeiffer_xlm_base/adapter_config.json
Module weights sav

In [34]:
trainer.push_to_hub('Training Completed')

Saving model checkpoint to ./de-adapter-32
Configuration saved in ./de-adapter-32/config.json
Model weights saved in ./de-adapter-32/pytorch_model.bin
tokenizer config file saved in ./de-adapter-32/tokenizer_config.json
Special tokens file saved in ./de-adapter-32/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.06G [00:00<?, ?B/s]

Upload file runs/May22_08-23-52_7cefe1b89645/events.out.tfevents.1653208006.7cefe1b89645.79.0:  44%|####3     …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/subhasisj/de-adapter-32
   ab85684..c0cbfbb  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/subhasisj/de-adapter-32
   c0cbfbb..954361d  main -> main



'https://huggingface.co/subhasisj/de-adapter-32/commit/c0cbfbbebb61a8f92e7497bfba646bef3a52d0ad'

In [ ]:
# trainer.save_model(f"./{language}-adapter-{batch_size}")

In [ ]:
# model.save_adapter('./hi-adapter-16/runs',"pfeiffer_xlm_base")

In [ ]:
# import os
# os.makedirs('./hi-adapter-16/all')

In [ ]:
# model.save_adapter('./hi-adapter-16/en-adapter',"en")

In [ ]:
# %%capture
# wandb.init(mode="disabled")
# wandb.init(mode="offline")

In [ ]:
# !zip -r   hi-adapters-16.zip  /kaggle/working/hi-adapter-16/all

In [ ]:
# import os
# os.chdir(r'/kaggle/working')
# from IPython.display import FileLink

# FileLink(r'./hi-adapters-16.zip')

In [ ]:
# !zip -r   hi-model-checkpoint-16.zip  /kaggle/working/hi-adapter-16/checkpoint-3500

In [ ]:
# FileLink(r'./hi-model-checkpoint-16.zip')